In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
splits = {'train': 'train_df.csv', 'validation': 'val_df.csv', 'test': 'test_df.csv'}
df_hf_multi_train = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["train"])
df_hf_multi_val = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["validation"])
df_hf_multi_test = pd.read_csv("hf://datasets/Sp1786/multiclass-sentiment-analysis-dataset/" + splits["test"])

In [3]:
# Combine all three datasets
df_hf_multi_all = pd.concat(
    [df_hf_multi_train, df_hf_multi_val, df_hf_multi_test],
    axis=0,        # Stack rows
    ignore_index=True  # Reset index
)

#print(df_hf_multi_all.shape)

#print(df_hf_multi_all.columns)

#print(df_hf_multi_all.isnull().sum())

# Display random sample
#df_hf_multi_all.sample(5)

#df_hf_multi_all['sentiment'].value_counts()


(41643, 4)
Index(['id', 'text', 'label', 'sentiment'], dtype='object')
id           0
text         1
label        0
sentiment    0
dtype: int64


sentiment
neutral     15507
positive    13968
negative    12168
Name: count, dtype: int64

In [4]:
df_hf_multi_all = df_hf_multi_all.dropna()
#print(df_hf_multi_all.isnull().sum())
#print(df_hf_multi_all.shape)
# Display random sample

id           0
text         0
label        0
sentiment    0
dtype: int64
(41642, 4)


In [5]:
df_hf_multi_all = df_hf_multi_all.drop(columns=['id'])
df_hf_multi_all = df_hf_multi_all.drop(columns=['sentiment'])
#display(df_hf_multi_all.head())
#get rid from redundant columns

,text,label
0,"Cooking microwave pizzas, yummy",2
1,Any plans of allowing sub tasks to show up in ...,1
2,"I love the humor, I just reworded it. Like sa...",2
3,naw idk what ur talkin about,1
4,That sucks to hear. I hate days like that,0


In [6]:
!pip install transformers datasets torch scikit-learn

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall:

2025-08-11 07:46:38.440718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754898398.644809      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754898398.704519      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=3, hidden_dropout_prob=0.3)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
#def tokenize_text(text):
#    return tokenizer(text, padding=True, truncation=True)

#df_hf_multi_all['tokenized'] = df_hf_multi_all['text'].apply(tokenize_text)


#df_hf_multi_all['tokenized']
#Wrong way. You should go through Hugging face


In [9]:

'''
# Convert lists to tensors
input_ids = torch.tensor(df_hf_multi_train['input_ids'].tolist())
attention_mask = torch.tensor(df_hf_multi_train['attention_mask'].tolist())
labels = torch.tensor(df_hf_multi_train['label'].tolist())  # if you have labels

# Now you can feed these into your model
outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)'''

"\n# Convert lists to tensors\ninput_ids = torch.tensor(df_hf_multi_train['input_ids'].tolist())\nattention_mask = torch.tensor(df_hf_multi_train['attention_mask'].tolist())\nlabels = torch.tensor(df_hf_multi_train['label'].tolist())  # if you have labels\n\n# Now you can feed these into your model\noutputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)"

In [10]:
from datasets import Dataset

# Rename label column if needed
df_hf_multi_all = df_hf_multi_all.rename(columns={'label': 'labels'})

# Convert to Dataset
dataset = Dataset.from_pandas(df_hf_multi_all)

In [11]:
#print(type(dataset))
#display(dataset[1])
#print(dataset)

In [12]:
def tokenize_function(example):
    return tokenizer(example['text'], padding=True, truncation=True)
    
tokenized_dataset = dataset.map(tokenize_function, batched=True)
#print(tokenized_dataset)
#print(tokenized_dataset[1])
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/41642 [00:00<?, ? examples/s]

In [13]:
#print(type(tokenized_dataset['input_ids']))


In [14]:
from transformers import DataCollatorWithPadding


class LoggingDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        max_len = batch['input_ids'].shape[1]
        print(f"📦 Batch padded to length: {max_len}")
        return batch #This all code so u could see max legth of each batch

data_collator = LoggingDataCollator(tokenizer=tokenizer)

In [15]:
#We will remove "text" column because it will affect memory usage and speed during training
#print(tokenized_dataset[1])

In [16]:

#Ask him can we do it our way(separete) like we did it in colab
# How this Validation is works at all? How it works on practise + examples of code? Is it used in the same time with training set durring training?
#Optionaly you could ask other questions like: AdamW, BertClassifier...
# Do not forget to delete "text" column from tokenized dataset before training

In [17]:
#The 80/10/10 split 
# Define len of train set
'''train_set_len = round(len(tokenized_dataset) * 0.8)
test_set_len = round(len(tokenized_dataset) * 0.1)
print(train_set_len)
print(test_set_len)

df = tokenized_dataset.to_pandas()

#Check is everything working
print(df.sample(5))
print(df.index)
token_copy = df.copy()
print(df.drop(1))
print("toooooooooken ---> ")
display(token_copy)
#print("tokenized_dataset - info")
tokenized_dataset
#print("tokenized_dataset - 1-st row")
#print(tokenized_dataset[1])

#Manual split of dataset. We used pandas dataframe for convenience.
# Divide by train and test set
train_set = df.sample(train_set_len) # Gives you 80% of Dataframe (random values)
display(train_set)

test_val_set = df.drop(train_set.index) # Get test and val sets
test_set_len = round(len(test_val_set) * 0.5)
test_set = test_val_set.sample(test_set_len)
val_set = test_val_set.drop(test_set.index)
#

from datasets import Dataset
#dataset = Dataset.from_pandas(df)
#test_set
train_set = Dataset.from_pandas(df)
train_set'''

'train_set_len = round(len(tokenized_dataset) * 0.8)\ntest_set_len = round(len(tokenized_dataset) * 0.1)\nprint(train_set_len)\nprint(test_set_len)\n\ndf = tokenized_dataset.to_pandas()\n\n#Check is everything working\nprint(df.sample(5))\nprint(df.index)\ntoken_copy = df.copy()\nprint(df.drop(1))\nprint("toooooooooken ---> ")\ndisplay(token_copy)\n#print("tokenized_dataset - info")\ntokenized_dataset\n#print("tokenized_dataset - 1-st row")\n#print(tokenized_dataset[1])\n\n#Manual split of dataset. We used pandas dataframe for convenience.\n# Divide by train and test set\ntrain_set = df.sample(train_set_len) # Gives you 80% of Dataframe (random values)\ndisplay(train_set)\n\ntest_val_set = df.drop(train_set.index) # Get test and val sets\ntest_set_len = round(len(test_val_set) * 0.5)\ntest_set = test_val_set.sample(test_set_len)\nval_set = test_val_set.drop(test_set.index)\n#\n\nfrom datasets import Dataset\n#dataset = Dataset.from_pandas(df)\n#test_set\ntrain_set = Dataset.from_pandas

In [18]:
# Checking how it works
from sklearn.model_selection import train_test_split

train_test_val = tokenized_dataset.train_test_split(test_size=0.2)
train_set = train_test_val['train']
#display(tokenized_dataset)
#display(train_test_val)
#display(train_set)

In [19]:
val_test_data = train_test_val['test']
test_val = val_test_data.train_test_split(test_size=0.5)
test_set = test_val["train"]
val_set = test_val["test"]

'''from sklearn.model_selection import train_test_split

# Step 1: Split into train+val and test
train_val_data, test_data = train_test_split(full_data, test_size=0.1, stratify=full_data['label'])

# Step 2: Split train+val into train and val
train_data, val_data = train_test_split(train_val_data, test_size=0.1111, stratify=train_val_data['label'])'''

"from sklearn.model_selection import train_test_split\n\n# Step 1: Split into train+val and test\ntrain_val_data, test_data = train_test_split(full_data, test_size=0.1, stratify=full_data['label'])\n\n# Step 2: Split train+val into train and val\ntrain_data, val_data = train_test_split(train_val_data, test_size=0.1111, stratify=train_val_data['label'])"

In [20]:
#display(test_set)

#display(val_set)

#print(tokenized_dataset[1])

#print(test_set[1])

#print(val_set[1])

#print(train_set[1])


In [21]:
#train_set.value_counts()
#test_set.value_counts()
#val_set.value_counts()
#This is not working because this method is for Pandas dataframes. We are using Hugging face datasets

from collections import Counter

from datasets import DatasetDict

# Example structure
dataset = DatasetDict({
    "train": train_set,
    "validation": val_set,
    "test": test_set
})




In [22]:
#display(dataset['train'])
#display(dataset['test'])
#display(dataset['validation'])

In [23]:
#print(dataset['train']['labels'][0])
#dataset['train']['labels'][:5]

#Counter(dataset['train']['labels'])


In [24]:
print(type(dataset['train']['labels']))


<class 'torch.Tensor'>


In [25]:
#from collections import Counter
'''
def count_labels(ds, label_column="labels"):
    return Counter(ds[label_column])

# Apply to each split
label_counts = {
    split: count_labels(dataset[split])
    for split in dataset
}

# Display results
for split, counts in label_counts.items():
    print(f"📊 {split.capitalize()} Label Counts:")
    for label, count in sorted(counts.items()):
        print(f"  Label {label}: {count}")'''

'\ndef count_labels(ds, label_column="labels"):\n    return Counter(ds[label_column])\n\n# Apply to each split\nlabel_counts = {\n    split: count_labels(dataset[split])\n    for split in dataset\n}\n\n# Display results\nfor split, counts in label_counts.items():\n    print(f"📊 {split.capitalize()} Label Counts:")\n    for label, count in sorted(counts.items()):\n        print(f"  Label {label}: {count}")'

Now, we check how many values for each label(negative, positive...) we have for each dataset. Are they evenly distributed

In [26]:
#labels = dataset['train']['labels']
#if isinstance(labels, torch.Tensor):
#    labels = labels.detach().cpu().numpy().tolist()

#label_counts = Counter(labels)
#print(label_counts)

In [27]:
#labels_test = dataset['test']['labels'].detach().cpu().numpy().tolist()

#label_counts_test = Counter(labels_test)
#print(label_counts_test)


In [28]:
#labels_val = dataset['validation']['labels'].detach().cpu().numpy().tolist()

#label_counts_val = Counter(labels_val)
#print(label_counts_val)

We deleting 'text' column for train dataset because it`s anyway will not be used durring training, but it will take memory and make speed slower

In [29]:
#display(dataset['train'])
dataset['train'] = dataset['train'].remove_columns(['text'])
#display(dataset['train'])

In [30]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [ ]:
def log_misclassifications(trainer, eval_dataset):
    predictions = trainer.predict(eval_dataset)
    preds = predictions.predictions.argmax(-1)
    labels = predictions.label_ids
    for i, (pred, label) in enumerate(zip(preds, labels)):
        if pred != label:
            print(f"[{i}] Pred: {pred}, True: {label}")

In [39]:
import transformers
from transformers import EarlyStoppingCallback
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

!pip install --upgrade transformers

print("Version of transformers: ")
print(transformers.__version__)

config = BertConfig.from_pretrained('bert-base-uncased', hidden_dropout_prob=0.3)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)



training_args = TrainingArguments(
    warmup_ratio=0.1, 
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='steps',
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type='cosine',  
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    fp16=True,
    save_total_limit=2,
    report_to='none'
    metric_for_best_model="eval_accuracy",  # or "eval_loss", "eval_f1", etc.
    greater_is_better=True  
)
# Here was mistake. TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'. Appeared that we just needed to change adviced by AI 'evaluation_strategy' on 'eval_strategy'. So instead of change it AI recomended to renew version of transformer. We almost stuck.

Version of transformers: 
4.52.4


In [33]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    data_collator=data_collator # Also added colletor for Dynamic Padding
)

In [34]:
trainer.train()

📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 416


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.588000,0.588142,0.754142,0.758158,0.760738,0.756163
2,0.477400,0.588071,0.757503,0.761737,0.764103,0.761040
3,0.379700,0.627841,0.758463,0.762143,0.762931,0.762062
4,0.348700,0.658794,0.756303,0.759865,0.760256,0.759608


📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 416
📦 Batch padded to length: 416
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 396
📦 Batch padded to length: 421
📦 Batch padded to length: 416
📦 Batch padded to length: 421
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 415
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 415
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch pa

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 415
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 415
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 415
📦 Batch padded to length: 421
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 396
📦 Batch pa

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 396
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 386
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 484
📦 Batch padded to length: 416
📦 Batch padded to length: 396
📦 Batch padded to length: 484
📦 Batch padded to length: 421
📦 Batch pa

TrainOutput(global_step=2084, training_loss=0.47751985264373603, metrics={'train_runtime': 6749.9387, 'train_samples_per_second': 19.741, 'train_steps_per_second': 0.309, 'total_flos': 3.105717912778043e+16, 'train_loss': 0.47751985264373603, 'epoch': 4.0})

In [38]:
from transformers import BertConfig, BertForSequenceClassification

config = BertConfig.from_pretrained('bert-base-uncased', num_labels=3, hidden_dropout_prob=0.3)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.save_model("sentiment-bert-final")
tokenizer.save_pretrained("sentiment-bert-final")

In [1]:
#if you would want to stop training - then ctrl + c

#if you would want to save your finetuned model
#trainer.save_model("best_model_checkpoint")
#tokenizer.save_pretrained("best_model_checkpoint")


#To reload it later: 
#from transformers import BertForSequenceClassification, BertTokenizer

#model = BertForSequenceClassification.from_pretrained("best_model_checkpoint")
#tokenizer = BertTokenizer.from_pretrained("best_model_checkpoint")

#---------------------------------
# Save model weights, config, and tokenizer
#trainer.save_model("sentiment-bert-final")
#tokenizer.save_pretrained("sentiment-bert-final")
#If you used load_best_model_at_end=True, then the current model in memory is already the best checkpoint. So saving it now will preserve that version.


#To use the model again
#from transformers import BertForSequenceClassification, BertTokenizer

#model = BertForSequenceClassification.from_pretrained("sentiment-bert-final")
#tokenizer = BertTokenizer.from_pretrained("sentiment-bert-final")

NameError: name 'trainer' is not defined

Test the fine-tuned model on the test set and evaluate accuracy using predefined metrics

In [ ]:
test_results = trainer.predict(dataset['test'])
print("Test Set Evaluation:")
for key, value in test_results.metrics.items():
    print(f"{key}: {value:.4f}")

In [ ]:
import numpy as np

preds = np.argmax(test_results.predictions, axis=-1)
labels = test_results.label_ids

for i in range(5):  # Show first 5 examples
    print(f"Predicted: {preds[i]}, True: {labels[i]}")